In [26]:
import pandas as pd

df_train = pd.read_csv('twitter_training.csv',sep=',',header=None)
df_val = pd.read_csv('twitter_validation.csv',sep = ',', header=None)

In [27]:
df = pd.concat([df_train.assign(ind="train"), df_val.assign(ind="validation")])

In [28]:
df =df.iloc[:,-3:]
df.columns = ['sentiment','text','ind']

In [29]:
df = df.loc[(df['sentiment'] == 'Positive') | (df['sentiment'] == 'Negative')]

In [5]:
df.sentiment.describe()

count        43917
unique           2
top       Negative
freq         22808
Name: sentiment, dtype: object

In [51]:
df.loc[df['sentiment'] == 'Positive','sentiment'] = 1
df.loc[df['sentiment'] == 'Negative','sentiment'] = 0

In [30]:
#pip install spacy
#in terminal: python -m spacy download en_core_web_sm

import spacy
nlp = spacy.load('en_core_web_sm')
import re

In [32]:
df["text"] = df["text"].astype(str)
df["text"] = df['text'].apply(lambda x: " ".join([y.lemma_ for y in nlp(x) if not y.is_stop]))
df['text'] = df['text'].apply(lambda x: x.lower())
df['text'] = df['text'].apply((lambda x: re.sub('[^\w\s]','',x)))

In [33]:
from tensorflow.keras.preprocessing.text import Tokenizer
from tensorflow.keras.preprocessing.sequence import pad_sequences

max_features = 3000
tokenizer = Tokenizer(num_words=max_features, split=' ')
tokenizer.fit_on_texts(df['text'].values)

In [52]:
df_train, df_val = df[df["ind"].eq("train")], df[df["ind"].eq("validation")]

In [53]:
X_train = tokenizer.texts_to_sequences(df_train['text'].values)
X_train = pad_sequences(X_train)

In [54]:
X_train.shape

(43374, 161)

In [55]:
X_val = tokenizer.texts_to_sequences(df_val['text'].values)
X_val = pad_sequences(X_val)

In [56]:
X_val.shape

(543, 28)

In [57]:
import numpy as np
X_val = np.lib.pad(X_val, ((0,0),(X_train.shape[1] - X_val.shape[1],0)), 'constant', constant_values=(0))
X_val.shape

(543, 161)

In [93]:
Y_train = np.array(pd.get_dummies((df_train['sentiment']).values))
Y_val = np.array(pd.get_dummies((df_val['sentiment']).values))

c:\Users\bruno\anaconda3\lib\site-packages\pandas\core\algorithms.py:798: FutureWarning: In a future version, the Index constructor will not infer numeric dtypes when passed object-dtype sequences (matching Series behavior)
  uniques = Index(uniques)


In [63]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Embedding, LSTM, SpatialDropout1D

In [66]:
embed_dim = 64
lstm_out = 128
max_features = 3000

model = Sequential()
model.add(Embedding(max_features, embed_dim,input_length = X_train.shape[1]))
model.add(SpatialDropout1D(0.15))
model.add(LSTM(lstm_out, dropout=0.15, recurrent_dropout=0.15))
model.add(Dense(2,activation='sigmoid'))
model.compile(loss = 'categorical_crossentropy', optimizer='adam',metrics = ['accuracy'])
print(model.summary())
batch_size = 16

Model: "sequential_1"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 161, 64)           192000    
                                                                 
 spatial_dropout1d_1 (Spatia  (None, 161, 64)          0         
 lDropout1D)                                                     
                                                                 
 lstm_1 (LSTM)               (None, 128)               98816     
                                                                 
 dense_1 (Dense)             (None, 2)                 258       
                                                                 
Total params: 291,074
Trainable params: 291,074
Non-trainable params: 0
_________________________________________________________________
None


In [67]:
model.fit(X_train, Y_train, epochs = 5, batch_size=batch_size, verbose = 3, shuffle=False)

Epoch 1/5
Epoch 2/5
Epoch 3/5
Epoch 4/5
Epoch 5/5


In [91]:
from sklearn.metrics import accuracy_score, f1_score, classification_report, confusion_matrix, roc_curve, auc

In [69]:
def evaluate_lstm(model, X_test,Y_test):

    pos_cnt, neg_cnt, pos_correct, neg_correct = 0, 0, 0, 0
    resultados = []

    for x in range(len(X_test)):
    
        result = model.predict(X_test[x].reshape(1,X_test.shape[1]),batch_size=1,verbose = 3)[0]
    
        if np.argmax(result) == np.argmax(X_test[x]):
            if np.argmax(X_test[x]) == 0:
                neg_correct += 1
            else:
                pos_correct += 1
        
        if np.argmax(X_test[x]) == 0:
            neg_cnt += 1
        else:
            pos_cnt += 1
        
        resultados.append(np.argmax(result))
    
    Y_test_argmax = np.argmax(Y_test,axis=1)
    Y_test_argmax  = Y_test_argmax.reshape(-1,1)
    resultados = np.asarray(resultados)
    resultados  = resultados.reshape(-1,1)

    conf_matrix = confusion_matrix(Y_test_argmax, classes_x)
    fig = plt.figure(figsize=(6, 6))
    sns.heatmap(conf_matrix, annot=True, fmt="d", cmap = 'GnBu');
    plt.title("Matriz de confusão ")
    plt.ylabel('Classe Correta')
    plt.xlabel('Classe Prevista')
 
    return accuracy_score(Y_test_argmax, resultados), f1_score(Y_test_argmax, resultados, pos_label=1), fig

In [70]:
accuracy,f1, fig = evaluate_lstm(model,X_val,Y_val)

In [71]:
print(accuracy)

0.7716390423572744


In [73]:
print(f1)

0.8103975535168195


In [ ]:
import matplotlib.pyplot as plt
import seaborn as sns

In [78]:
evaluate_lstm(model,X_val,Y_val)

In [99]:
X_val.shape,Y_val.shape

((543, 161), (543, 2))

In [104]:
predict_x=model.predict(X_val) 
classes_x=np.argmax(predict_x,axis=1)
print(predict_x)
print(classes_x)
print(Y_val)
y_test_max=np.argmax(Y_val) 
classes_x = classes_x.reshape(-1,1)
print(y_test_max)
conf_matrix = confusion_matrix(y_test_max, classes_x)
fig = plt.figure(figsize=(6, 6))
sns.heatmap(conf_matrix, annot=True, fmt="d", cmap = 'GnBu');
plt.title("Matriz de confusão ")
plt.ylabel('Classe Correta')
plt.xlabel('Classe Prevista')

17/17 [==============================] - 1s 33ms/step
[[0.8533949  0.16308916]
 [0.45998907 0.4999645 ]
 [0.93655074 0.043003  ]
 ...
 [0.28881276 0.7159194 ]
 [0.08779088 0.83670086]
 [0.08358266 0.9227474 ]]
[0 1 0 1 1 0 1 1 1 0 0 1 1 1 1 0 1 0 0 0 1 1 1 1 0 1 1 1 1 0 1 1 0 0 1 1 1
 1 1 0 1 1 1 0 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 0 0 1 1 1 1
 1 1 0 0 1 1 1 1 1 1 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 0 0 1 1 1 1 0 0 1 0 1 1
 1 1 1 0 1 1 1 0 0 1 1 0 1 1 1 0 1 1 0 1 1 1 0 1 1 1 1 0 1 1 1 1 1 0 1 1 0
 1 1 0 1 1 0 1 1 0 0 1 1 1 1 0 1 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 1 1 1 0 0 1 0 1 1 0 1 1 0 1 0 1 1 1 0 1 0 1 0 1 1 1 1 1 1 1 0 1 1
 1 1 0 0 1 1 0 1 1 1 1 1 1 0 0 0 0 1 1 1 1 1 0 0 1 0 1 1 1 1 0 1 0 0 1 1 0
 0 1 1 0 1 1 1 1 0 0 1 1 1 1 1 1 0 1 0 1 1 0 1 1 1 0 0 0 1 1 1 0 0 0 1 0 1
 1 0 1 0 1 1 1 1 0 0 0 1 1 0 1 0 0 0 1 0 1 0 0 0 0 1 1 1 1 1 1 1 0 1 0 1 1
 1 1 1 1 1 1 1 0 1 0 0 1 1 0 0 0 1 1 1 0 1 1 0 1 1 1 1 0 1 1 1 1 1 1 1 0 1
 1 0 1 1 1 1 0 1 1 1 1 1 1 1 0 0 1 1 0 1

TypeError: Singleton array 0 cannot be considered a valid collection.